# Summarize Dialogue (GPT-4)

<a name='1'></a>
## 1 - Set up Kernel and Required Dependencies

In [1]:
# # Upgrade pip, setuptools, and wheel to prevent version conflicts
# %pip install --upgrade pip setuptools wheel --quiet

# # Install all required packages in a single command for efficiency
# %pip install --no-cache-dir --quiet \
#     tensorflow keras torchdata datasets evaluate rouge_score peft

In [ ]:
import os
from datasets import load_dataset
import openai
from dotenv import load_dotenv
import general_util


## 2 - Loading of test data from Huggingface

In [4]:
# Load dataset from huggingface
dataset = general_util.huggingface_dataset()

## 3 - Output selected test data used

In [5]:
# Printing of samples indicies
sample_indices = [40, 80, 120, 200]
general_util.display_dialogue_summaries(sample_indices, dataset)

----------------------------------------------------------------------------------------------------
Example 1
----------------------------------------------------------------------------------------------------
INPUT DIALOGUE:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
----------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
----------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------

In [6]:
def generate_openai_summary(input_text: str, model_name: str = "gpt-4") -> str:
    """
    Generates a summary using OpenAI's GPT-4 model.

    Args:
        input_text (str): The input prompt for the model.
        model_name (str): The OpenAI model to use (default is "gpt-4").

    Returns:
        str: The generated summary from OpenAI.
    """
    # Load environment variables from .env file
    load_dotenv()

    # Retrieve API key
    openai_api_key = os.getenv("OPENAI_API_KEY")

    # Ensure API key is loaded
    if not openai_api_key:
        raise ValueError("❌ Error: OPENAI_API_KEY is not set. Check your .env file!")

    # Set API key
    openai.api_key = openai_api_key

    # Generate response
    response = openai.chat.completions.create(
        model=model_name,
        messages=[{"role": "user", "content": input_text}]
    )

    # Extract and return the generated summary
    return response.choices[0].message.content

## 4 - Output of Test Result

### 4.1 - Zero-Short Inference

In [7]:
# Initialize Instruction 
instruction = "Generate a summary based on the above Dialogue"

# Initialize an empty list to store training indices (for few-shot learning if needed)
list_index_to_train = []

# Set the index of the dialogue to summarize (zero-shot inference)
index_to_summarize = 40

# Generate the input prompt for the OpenAI model using a utility function
input_text, summary = general_util.make_prompt(dataset, instruction, index_to_summarize, list_index_to_train)

# Generate a summary using OpenAI's GPT-4 model
generated_summary = generate_openai_summary(input_text, "gpt-4")

# Output the results (label, ground truth summary, generated summary)
general_util.output_text("ZERO SHOT", summary, generated_summary)

----------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.

----------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
Tom tells the first person that it's almost nine o'clock. Surprised, the first person realizes they need to leave to catch the nine-thirty train. Tom assures them there's no rush as the railway station is close and the journey should only take twenty minutes.



### 4.2 - One-Short Inference

In [11]:
list_index_to_train = [20]
index_to_summarize = 40
input_text, summary = general_util.make_prompt(dataset, instruction, index_to_summarize, list_index_to_train)
generated_summary = generate_openai_summary(input_text, "gpt-4")
general_util.output_text("ONE SHOT", summary, generated_summary)

----------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.

----------------------------------------------------------------------------------------------------
MODEL GENERATION - ONE SHOT:
#Person1# realizes it's almost nine and leaves to catch the nine-thirty train. #Person2# reassures #Person1# that there is plenty of time as the railway station is nearby.



### 4.3 - Few-Short Inference

In [9]:
list_index_to_train = [200, 80, 120]
index_to_summarize = 40
input_text, summary = general_util.make_prompt(dataset, instruction, index_to_summarize, list_index_to_train)
generated_summary = generate_openai_summary(input_text, "gpt-4")
general_util.output_text("FEW SHOT", summary, generated_summary)

----------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.

----------------------------------------------------------------------------------------------------
MODEL GENERATION - FEW SHOT:
#Person1# is worried about the time as they need to catch the nine-thirty train, but #Person2# assures them that they have plenty of time as the railway station is close by.

